In [1]:
lat1 = 0.0
lat2 = 10.0
lon1 = 0.0
lon2 = 40.0
time1 = "2017-07-01T00:00:00"
time2 = "2017-07-01T2:59:59"
gstpFilter = c3.Filter() \
    .ge("latitude", lat1) \
    .and_().le("latitude", lat2) \
    .and_().ge("longitude", lon1) \
    .and_().le("longitude", lon2) \
    .and_().ge("time", time1) \
    .and_().le("time", time2)

excludeFeats = ["acure_anth_so2", "acure_carb_bb_ems", "acure_carb_ff_ems", "acure_carb_res_ems"]
kernelLen = 59 - len(excludeFeats)

GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*kernelLen, nu=0.5, coefficient=1.0).build().kernel

GPR_technique = c3.GaussianProcessRegressionTechnique(
                    randomState=42,
                    kernel = GPR_kernel
)

In [2]:
nFeatures = kernelLen

# create synthetic data with numpy
import numpy as np

synth = np.random.rand(50, nFeatures)

# cast it into a c3.Dataset
synthDataset = c3.Dataset.fromPython(pythonData=synth)

In [3]:
job = c3.AODGPRModelFinder.extractLearnedParametersJob(excludeFeats, gstpFilter, "all", GPR_technique, 10)

In [10]:
job.status()

c3.MapReduceStatus(
 started=datetime.datetime(2022, 11, 8, 21, 36, 40, tzinfo=datetime.timezone.utc),
 startedby='jcarzon@andrew.cmu.edu',
 completed=datetime.datetime(2022, 11, 8, 21, 41, 33, tzinfo=datetime.timezone.utc),
 status='failed',
 errors=c3.Arry<JobRunErrorDetail>([c3.JobRunErrorDetail(
           failedActionId='3514.16420983',
           errorMsg='wrapped IOException: Broken pipe\n'
                     '\n'
                     '    at java.io.FileOutputStream.writeBytes\n'
                     '    ...\n'
                     '    at '
                     'c3.love.typesys.obj.pipe.ser.JsonObjValueReceiver.writePrimitive '
                     '(JsonObjValueReceiver.java:518)\n'
                     '\n'
                     '  from action Lambda<function(key: any, interValues: '
                     'any, job: any): any>.call\n'
                     '  from env_server.js, line 132\n'
                     '    130    if (C3._context.locale)\n'
                     '   

In [ ]:
df = c3.AODGPRModelFinder.getDataframeFromJob(job)

In [ ]:
df

In [3]:
job2 = c3.PredictAODGPR.makePredictionsJob(excludeFeats, gstpFilter, "all", synthDataset, GPR_technique, 10)

In [6]:
job2.status()

c3.MapReduceStatus(
 started=datetime.datetime(2022, 11, 7, 21, 32, 3, tzinfo=datetime.timezone.utc),
 startedby='jcarzon@andrew.cmu.edu',
 completed=datetime.datetime(2022, 11, 7, 21, 35, 37, tzinfo=datetime.timezone.utc),
 status='completed')

In [ ]:
print(job2.status().errors[0].errorMsg)

In [ ]:
import pandas as pd

predictions = []

if job2.status().status == "completed":
    for key, value in job2.results().items():
        for subvalue in value:
            df_m = pd.DataFrame()
            df_m["mean"] = np.array(subvalue[0]).flatten()
            df_m["mean"] += subvalue[3]
            df_m["sd"] = subvalue[1]
            df_m["modelId"] = subvalue[2]

            predictions.append(df_m)

#df = pd.concat(predictions, axis=0).reset_index(drop=True)
predictions

In [ ]:
if job2.status().status == "completed":
    for key, value in job2.results().items():
        for subvalue in value:
            df_m = pd.DataFrame()
            df_m["mean"] = np.array(subvalue[0]).flatten()
            print(df_m)

In [7]:
df2 = c3.PredictAODGPR.getPredictionsDataframeFromJob(job2)

In [8]:
df2

,mean,sd,modelId
0,0.324550,0.083335,6937e341-13da-447f-8ed9-3b33b6cdcf8c
1,0.620296,0.074768,6937e341-13da-447f-8ed9-3b33b6cdcf8c
2,0.377728,0.081698,6937e341-13da-447f-8ed9-3b33b6cdcf8c
3,0.590834,0.075627,6937e341-13da-447f-8ed9-3b33b6cdcf8c
4,0.339952,0.080818,6937e341-13da-447f-8ed9-3b33b6cdcf8c
...,...,...,...
845,0.825088,0.072095,e1b530e6-cb7b-4ffe-b360-e4b451a3e579
846,0.384072,0.065991,e1b530e6-cb7b-4ffe-b360-e4b451a3e579
847,0.550648,0.069448,e1b530e6-cb7b-4ffe-b360-e4b451a3e579
848,0.472630,0.068938,e1b530e6-cb7b-4ffe-b360-e4b451a3e579


In [ ]:
# Put SynthDataset in job context

# Get parameters

## Og function

In [ ]:
def extractLearnedParametersJob(
    excFeats, gstpFilter, targetName, technique, batchSize
):
    """
    Dynamic map-reduce job to extract learned hyper parameters.
    """
    def cassandra_mapper(batch, objs, job):
        models = []
        for obj in objs:
            model = c3.AODGPRModelFinder.getPipe(
                job.context.value["excludeFeatures"],
                obj.id,
                job.context.value["targetName"],
                job.context.value["technique"]
            )
            models.append(model)
        
        return {batch: models}

    def cassandra_reducer(key, interValues, job):
        values = []
        for iv in interValues:
            for val in iv:
                for m in val:
                    pickledModel = m["trainedModel"]["model"]
                    model = c3.PythonSerialization.deserialize(serialized=pickledModel)
                    hp = model.kernel_.get_params()['k2__length_scale']
                    model_id = m["id"]

                    # find GSTP
                    dssId = m["dataSourceSpec"]["id"]
                    dss = c3.GPRDataSourceSpec.get(dssId)
                    gstpId = dss.targetSpec.filter.split(" == ")[1].replace('"', '')
                    gstp = c3.GeoSurfaceTimePoint.get(gstpId)
                    lat = gstp.latitude
                    lon = gstp.longitude
                    time = gstp.time
                    values.append((hp, model_id, lat, lon, time))

        return values

    map_lambda = c3.Lambda.fromPython(cassandra_mapper)
    reduce_lambda = c3.Lambda.fromPython(cassandra_reducer, runtime="gordon-ML_1_0_0")

    job_context = c3.MappObj(
        value={
            'excludeFeatures': excFeats,
            'targetName': targetName,
            'technique': technique
        }
    )
    job = c3.DynMapReduce.startFromSpec(
        c3.DynMapReduceSpec(
            targetType="GeoSurfaceTimePoint",       
            filter=gstpFilter, 
            mapLambda=map_lambda,
            reduceLambda=reduce_lambda,
            batchSize=batchSize,
            context=job_context
        )
    )

    return job

## Proposed functions

In [ ]:
def makePredictionsJob(
    excFeats, gstpFilter, targetName, synthDataset, technique, batchSize
):
    """
    Dynamic map-reduce job to get predictions on synthDataset.
    """
    
    def cassandra_mapper(batch, objs, job):
        models = []
        for obj in objs:
            model = c3.AODGPRModelFinder.getPipe(
                job.context.value["excludeFeatures"],
                obj.id,
                job.context.value["targetName"],
                job.context.value["technique"]
            )
            models.append(model)
        
        return {batch: models}

    def cassandra_reducer(key, interValues, job):
        values = []
        synthDataframe = c3.Dataset.toPandas(synthDataset)

        for iv in interValues:
            for val in iv:
                for m in val:

                    model_id = m["id"]

                    centered = m["technique"]["centerTarget"]
                    if centered:
                        center = m["trainedModel"].parameters["targetMean"].asfloat()
                    else:
                        center = 0

                    pickledModel = m["trainedModel"]["model"]
                    model = c3.PythonSerialization.deserialize(serialized=pickledModel)
                    mean, sd = model.predict(synthDatarame, return_std=True)
                    preds = pd.concat([pd.DataFrame(mean), pd.DataFrame(sd)], axis=1)

                    values.append((preds, synthDataset, model_id, center))
                    

        return values

    map_lambda = c3.Lambda.fromPython(cassandra_mapper)
    reduce_lambda = c3.Lambda.fromPython(cassandra_reducer, runtime="gordon-ML_1_0_0")

    job_context = c3.MappObj(
        value={
            'excludeFeatures': excFeats,
            'targetName': targetName,
            'technique': technique,
            'syntheticDataset': synthDataset
        }
    )
    job = c3.DynMapReduce.startFromSpec(
        c3.DynMapReduceSpec(
            targetType="GeoSurfaceTimePoint",       
            filter=gstpFilter, 
            mapLambda=map_lambda,
            reduceLambda=reduce_lambda,
            batchSize=batchSize,
            context=job_context
        )
    )

    return job

## Break apart

In [ ]:
pipe = c3.GaussianProcessRegressionPipe.fetch({
    "limit": 1 
}).objs[0]

filt = c3.Filter().intersects("technique.id", [pipe.technique.id]).and_().intersects("dataSourceSpec.id", [pipe.dataSourceSpec.id])
pipes = c3.GaussianProcessRegressionPipe.fetch({"filter": filt.value, "limit": -1})
objs = pipes.objs

In [ ]:
m = objs[0]

In [ ]:
sourceSpec = c3.GPRDataSourceSpec.get(pipe.dataSourceSpec.id)
nExcFeats = len(sourceSpec.excludeFeatures)
nFeatures = 59 - nExcFeats

In [ ]:
nFeatures

In [ ]:
# create synthetic data with numpy
import numpy as np

synth = np.random.rand(50, nFeatures)

# cast it into a c3.Dataset
synthDataset = c3.Dataset.fromPython(pythonData=synth)

In [ ]:
for m in objs:
    preds = m.process(synthDataset)
    
    

In [ ]:
preds

In [ ]:
trainedModel.parameters['targetMean']

In [ ]:
pickledModel = m.trainedModel.model
model = c3.PythonSerialization.deserialize(serialized=pickledModel)

In [ ]:
mean, sd = model.predict(c3.Dataset.toPandas(synthDataset), return_std=True)

In [ ]:
pd.concat([pd.DataFrame(mean), pd.DataFrame(sd)], axis=1)

In [ ]:
def cassandra_mapper(batch, objs, job):
    models = []
    for obj in objs:
        model = c3.AODGPRModelFinder.getPipe(
            job.context.value["excludeFeatures"],
            obj.id,
            job.context.value["targetName"],
            job.context.value["technique"]
        )
        models.append(model)

    return {batch: models}

In [ ]:
def cassandra_reducer(key, interValues, job):
    values = []
    for iv in interValues:
        for val in iv:
            for m in val:
                pickledModel = m["trainedModel"]["model"]
                model = c3.PythonSerialization.deserialize(serialized=pickledModel)
                hp = model.kernel_.get_params()['k2__length_scale']
                model_id = m["id"]

                # find GSTP
                dssId = m["dataSourceSpec"]["id"]
                dss = c3.GPRDataSourceSpec.get(dssId)
                gstpId = dss.targetSpec.filter.split(" == ")[1].replace('"', '')
                gstp = c3.GeoSurfaceTimePoint.get(gstpId)
                lat = gstp.latitude
                lon = gstp.longitude
                time = gstp.time
                values.append((hp, model_id, lat, lon, time))

    return values

In [ ]:
map_lambda = c3.Lambda.fromPython(cassandra_mapper)
reduce_lambda = c3.Lambda.fromPython(cassandra_reducer, runtime="gordon-ML_1_0_0")

In [ ]:
job_context = c3.MappObj(
    value={
        'excludeFeatures': excludeFeats,
        'targetName': "all",
        'technique': GPR_technique
    }
)

In [ ]:
job = c3.DynMapReduce.startFromSpec(
    c3.DynMapReduceSpec(
        targetType="GeoSurfaceTimePoint",       
        filter=myFilter, 
        mapLambda=map_lambda,
        reduceLambda=reduce_lambda,
        batchSize=10,
        context=job_context
    )
)

In [ ]:
print(job.status().errors[0].errorMsg)

In [ ]:
sourceSpec = c3.GPRDataSourceSpec.get(pipe.dataSourceSpec.id)
excFeats = sourceSpec.excludeFeatures

In [ ]:
targetName = sourceSpec.targetName

In [ ]:
gstpId = "-21.875_19.688_2017-07-28T18:20:00"

In [ ]:
gprTechnique = c3.GaussianProcessRegressionTechnique.get(pipe.technique.id)

In [ ]:
c3.SklearnGPRKernel.get(gprTechnique.kernel.id).pickledKernel

In [ ]:
c3.AODGPRModelFinder.getPipe(
    excFeats,
    gstpId,
    targetName,
    gprTechnique
)

In [ ]:
GPR_technique = c3.GaussianProcessRegressionTechnique.get('BHE7')

In [ ]:
myJob = extractLearnedParametersJob(
    excludeFeats,
    myFilter,
    "all",
    GPR_technique,
    10
)

In [ ]:
myJob.cancel()

# Dataframe

## Og function

In [ ]:
def getDataframeFromJob(job):
    """
    Iterates over job result and builds dataframe.
    """
    import pandas as pd
    import numpy as np

    lengthScales = []
    ids = []
    lats = []
    lons = []
    times = []
    if job.status().status == "completed":
        for key, value in job.results().items():
            for subvalue in value:
                ls = np.array(subvalue[0]).astype(float)
                model_id = np.array([subvalue[1]]).astype(str)
                lengthScales.append(ls)
                ids.append(model_id[0])
                lats.append(np.array(subvalue[2]).astype(float))
                lons.append(np.array(subvalue[3]).astype(float))
                times.append(np.array(subvalue[4]))
                
        df = pd.DataFrame(lengthScales)
        df["modelId"] = ids
        df["latitude"] = lats
        df["longitude"] = lons
        df["time"] = times
        return df
    else:
        return False

## Proposed function

In [ ]:
def getPredictionsDataframeFromJob(job):
    """
    Iterates over job result and builds dataframe.
    """
    import pandas as pd
    import numpy as np

    predictions = []
    
    if job.status().status == "completed":
        for key, value in job.results().items():
            for subvalue in value:
                df_y = c3.Dataset.toPandas(subvalue[0])
                df_y[0] += subvalue[3]
                df_x = c3.Dataset.toPandas(subvalue[1])
                m_preds = pd.concat(
                    [df_x, df_y],
                    axis=1
                )
                m_preds["modelId"] = subvalue[2]
                predictions.append(m_preds)
                
        df = pd.concat(predictions, axis=0).reset_index(drop=True)
        return df
    else:
        return False

In [ ]:
getDataframeFromJob(myJob)